In [1]:
import pandas as pd
import numpy  as np

import pickle
  
from scipy.sparse import issparse 
 
from pathlib import Path


pd.set_option('display.max_columns', 5)

In [2]:
#### user specified
data_folder = r"D:/analyze_Pearson_residuals/"

data_subfolder = "Macosko_GEO"

data_path = Path ( data_folder + data_subfolder )

In [3]:
# output data
results_pkl = "residual_variance_scanpy.pkl"

# input data
counts_scipy_csc_pkl  = "counts_scipy_csc.pkl"
gene_array_pkl  = "gene_array.pkl"

 
# path: output data
results_dsn = data_path / results_pkl

# paths: input data
counts_scipy_csc_dsn = data_path / counts_scipy_csc_pkl
gene_array_dsn = data_path / gene_array_pkl 

In [4]:
# https://github.com/scverse/scanpy/tree/master/scanpy/experimental/pp
# from   _normalization.py  rows 26-64

def _pearson_residuals(X, theta, clip, copy=False):    # removed check_values from parm list

    X = X.copy() if copy else X

    # check theta
    if theta <= 0:
        # TODO: would "underdispersion" with negative theta make sense?
        # then only theta=0 were undefined..
        raise ValueError('Pearson residuals require theta > 0')
    # prepare clipping
    if clip is None:
        n = X.shape[0]
        clip = np.sqrt(n)
    if clip < 0:
        raise ValueError("Pearson residuals require `clip>=0` or `clip=None`.")

# vfk 2022 08 16		
    # if check_values and not check_nonnegative_integers(X):
        # warn(
            # "`normalize_pearson_residuals()` expects raw count data, but non-integers were found.",
            # UserWarning,
        # )

    if issparse(X):
        sums_genes = np.sum(X, axis=0)
        sums_cells = np.sum(X, axis=1)
        sum_total = np.sum(sums_genes).squeeze()
    else:
        sums_genes = np.sum(X, axis=0, keepdims=True)
        sums_cells = np.sum(X, axis=1, keepdims=True)
        sum_total = np.sum(sums_genes)

    mu = np.array(sums_cells @ sums_genes / sum_total)
    diff = np.array(X - mu)
    residuals = diff / np.sqrt(mu + mu**2 / theta)

    # clip
    residuals = np.clip(residuals, a_min=-clip, a_max=clip)

    return residuals	
	
	  
	
pctl_list = [.01,.05, .10, .25, .5, .75, .90, .95, .96, .97, .98, .99 ]

In [5]:
f = open( counts_scipy_csc_dsn, 'rb' )  
scipy_csc_mat = pickle.load( f )           
f.close()       
print ( ' scipy_csc_mat.shape: ' ,  scipy_csc_mat.shape )
print ( '\n scipy_csc_mat:\n' , scipy_csc_mat)
 
scipy_csr_mat = scipy_csc_mat.transpose().astype ( 'float64'  )
print ( '\n\n scipy_csr_mat.shape: ' ,  scipy_csr_mat.shape )
print ( '\n scipy_csr_mat:\n' , scipy_csr_mat)

del scipy_csc_mat  


f = open( gene_array_dsn, 'rb' )
gene_array = pickle.load( f )           
f.close()       
print ( '\n\n  gene_array.shape:  ' ,  gene_array.shape )
print ( '\n gene_array: ' , gene_array)

 scipy_csc_mat.shape:  (13552, 24769)

 scipy_csc_mat:
   (2, 0)	496
  (4, 0)	7
  (5, 0)	69
  (6, 0)	471
  (8, 0)	131
  (9, 0)	11
  (11, 0)	415
  (12, 0)	2
  (14, 0)	6
  (15, 0)	1
  (21, 0)	2
  (26, 0)	170
  (27, 0)	254
  (28, 0)	2
  (31, 0)	296
  (32, 0)	1
  (33, 0)	1
  (35, 0)	3
  (37, 0)	93
  (38, 0)	203
  (40, 0)	1
  (42, 0)	2
  (45, 0)	205
  (49, 0)	2
  (50, 0)	5
  :	:
  (8417, 24768)	1
  (8646, 24768)	1
  (8648, 24768)	1
  (8658, 24768)	1
  (8724, 24768)	1
  (8747, 24768)	1
  (8927, 24768)	1
  (8932, 24768)	1
  (8999, 24768)	1
  (9006, 24768)	1
  (9008, 24768)	1
  (9245, 24768)	1
  (9487, 24768)	2
  (9607, 24768)	1
  (9881, 24768)	1
  (9936, 24768)	1
  (9946, 24768)	1
  (9950, 24768)	1
  (10068, 24768)	1
  (10137, 24768)	1
  (10139, 24768)	1
  (10338, 24768)	1
  (10435, 24768)	1
  (11428, 24768)	1
  (11460, 24768)	1


 scipy_csr_mat.shape:  (24769, 13552)

 scipy_csr_mat:
   (0, 2)	496.0
  (0, 4)	7.0
  (0, 5)	69.0
  (0, 6)	471.0
  (0, 8)	131.0
  (0, 9)	11.0
  (0, 11)	415.0
  (0, 

In [6]:
residuals = _pearson_residuals ( scipy_csr_mat, 100, 1e9 )
residual_variance_unclipped = np.var( residuals, axis=0)

residuals = _pearson_residuals ( scipy_csr_mat, 100, None )
residual_variance_clipped = np.var( residuals, axis=0)

In [7]:
df_results = pd.DataFrame ( index = gene_array, data = { 'CLIPPED': residual_variance_clipped, 'UNCLIPPED': residual_variance_unclipped } ) 
print ( '\n df_results' )
print ( df_results  )
print ( '\n\n df_results.describe' )
print ( df_results.describe ( percentiles=pctl_list ) )


 df_results
                 CLIPPED  UNCLIPPED
CARTPT         44.270155  49.332877
RGS5           20.431153  21.796726
RHO            12.699057  12.699057
APOE           16.193743  16.193743
GLUL           15.138930  15.138930
...                  ...        ...
RTN4R           0.553089   0.553089
GM25749         0.425352   0.425352
KIF26B          0.504509   0.504509
A330033J07RIK   0.337237   0.337237
RHOV            0.390322   0.390322

[13552 rows x 2 columns]


 df_results.describe
            CLIPPED     UNCLIPPED
count  13552.000000  13552.000000
mean       1.215406      1.223827
std        0.727920      1.022937
min        0.337237      0.337237
1%         0.633936      0.633936
5%         0.768041      0.768041
10%        0.852360      0.852360
25%        1.007559      1.007559
50%        1.141105      1.141105
75%        1.296523      1.296523
90%        1.492995      1.492995
95%        1.671891      1.671891
96%        1.741279      1.741279
97%        1.886250      1.886

In [8]:
df_results.to_pickle ( results_dsn )